Original Author: [Kaggle](https://www.kaggle.com/code/terencicp/steam-games-data-transformation/notebook)

1. Steam Games Dataset by Martin Bustos
2. Video Games on Steam by Sujay Kapadnis

In [ ]:
import pandas as pd
import json
import numpy as np

And load the first file:

In [46]:
first_file = 'games.json'
with open(first_file, 'r', encoding='utf-8') as file:
    json_data = json.load(file)

I'll convert the JSON into a DataFrame, ignoring the variables I won't analyze:

In [47]:
# Variables I won't analyze
unnecessary_vars = [
    'packages', 'screenshots', 'movies', 'score_rank', 'header_image',
    'reviews', 'website', 'support_url', 'notes', 'support_email',
    'recommendations', 'user_score', 'median_playtime_forever',
    'median_playtime_2weeks', 'required_age', 'metacritic_score',
    'metacritic_url', 'peak_ccu', 'detailed_description', 'about_the_game',
    'windows', 'mac', 'linux', 'achievements', 'full_audio_languages',
    'genres', 'dlc_count', 'supported_languages', 'developers',
    'publishers', 'average_playtime_forever', 'average_playtime_2weeks'
]

In [48]:
# Process each game's information and store in a list
games = [{
    **{k: v for k, v in game_info.items() if k not in unnecessary_vars},
    'tags': list(tags.keys()) if isinstance((tags := game_info.get('tags', {})), dict) else [],
    'tag_frequencies': list(tags.values()) if isinstance(tags, dict) else [],
    'app_id': app_id
} for app_id, game_info in json_data.items()]

# Create a DataFrame from the processed list
df = pd.DataFrame(games)

Let's have a look at the DataFrame:

In [49]:
df

,name,release_date,price,short_description,categories,positive,negative,estimated_owners,tags,tag_frequencies,app_id
0,Counter-Strike 2,2012-08-21,0.00,"For over two decades, Counter-Strike has offer...","[['Multi-player', 'Cross-Platform Multiplayer'...",7024836,1029779,100000000 - 200000000,"[FPS, Shooter, Multiplayer, Competitive, Actio...","[90076, 64786, 61798, 52887, 47165, 46061, 431...",730
1,PUBG: BATTLEGROUNDS,2017-12-21,0.00,Play PUBG: BATTLEGROUNDS for free. Land on str...,"[['Multi-player', 'PvP', 'Online PvP', 'Stats'...",1379580,981860,50000000 - 100000000,"[Survival, Shooter, Battle Royale, Multiplayer...","[14611, 12477, 10683, 10648, 8242, 7598, 7202,...",578080
2,Dota 2,2013-07-09,0.00,"Every day, millions of players worldwide enter...","[['Multi-player', 'Co-op', 'Steam Trading Card...",1832477,406030,200000000 - 500000000,"[Free to Play, MOBA, Multiplayer, Strategy, e-...","[59559, 19966, 15153, 14096, 11631, 10822, 816...",570
3,Grand Theft Auto V,2015-04-13,0.00,Grand Theft Auto V for PC offers players the o...,"[['Single-player', 'Multi-player', 'PvP', 'Onl...",1557234,236827,50000000 - 100000000,"[Open World, Action, Multiplayer, Crime, Autom...","[31998, 23177, 21328, 19059, 18579, 18412, 180...",271590
4,Tom Clancy's Rainbow Six® Siege,2015-12-01,19.99,"Tom Clancy's Rainbow Six® Siege is an elite, t...","[['Single-player', 'Multi-player', 'PvP', 'Onl...",1094330,190046,20000000 - 50000000,"[FPS, PvP, e-sports, Multiplayer, Tactical, Sh...","[9689, 9073, 9026, 8948, 8948, 8944, 8901, 888...",359550
...,...,...,...,...,...,...,...,...,...,...,...
83641,Circuit Breaker,2021-10-21,4.99,"In the cyber-future of 2009, the most popular ...","[['Single-player', 'Family Sharing']]",1,0,0 - 20000,"[Strategy, Casual, PvE, Singleplayer, Tactical...","[72, 48, 47, 44, 41, 37, 32, 29, 27, 25, 22, 2...",1745960
83642,Chess vs Chat,2022-08-09,3.99,Allow your chatters to actively participate an...,"[['Single-player', 'Multi-player', 'PvP', 'Onl...",2,1,0 - 20000,"[Indie, Strategy, Puzzle, Multiplayer, 2D, Che...","[31, 21, 11, 11, 11, 11, 11, 11]",1888920
83643,Orbtangle,2022-10-28,1.99,Orbtangle is a simplistic puzzle game which re...,"[['Single-player', 'Steam Achievements', 'Fami...",5,0,0 - 20000,"[Puzzle, Colorful, Relaxing, Logic, Minimalist...","[54, 46, 44, 41, 38, 33, 30, 27, 25, 24, 22, 2...",2166420
83644,Flashlight,2021-10-25,2.99,"Salvage scrap, build defenses, and rank-up to ...","[['Single-player', 'Multi-player', 'PvP', 'Onl...",3,0,0 - 20000,"[Top-Down Shooter, Rogue-lite, Action Roguelik...","[138, 132, 128, 125, 121, 116, 108, 107, 104, ...",1456260


## Data Cleaning

Great! We've got the data we need, but it seems we have more rows than there are games on Steam. Let's see how many games have no sales:

In [50]:
count = (df['estimated_owners'] == "0 - 0").sum()
print("Number of games with estimated owners '0 - 0':", count)

Number of games with estimated owners '0 - 0': 6573


In [51]:
df[df['estimated_owners'] == "0 - 0"]

,name,release_date,price,short_description,categories,positive,negative,estimated_owners,tags,tag_frequencies,app_id
44,Overwatch® 2,2023-08-10,0.00,"Overwatch 2 is a critically acclaimed, team-ba...","[['Multi-player', 'PvP', 'Online PvP', 'Co-op'...",0,0,0 - 0,[],[],2357570
102,THE FINALS,2023-12-07,0.00,"Join THE FINALS, the world-famous, free-to-pla...","[['Multi-player', 'PvP', 'Online PvP', 'Co-op'...",0,0,0 - 0,[],[],2073850
104,Muck,2021-06-05,0.00,Muck is a survival-roguelike. Collect resource...,"[['Single-player', 'Multi-player', 'PvP', 'Onl...",0,0,0 - 0,[],[],1625450
114,Crab Game,2021-10-29,0.00,Crab Game is a First-Person Multiplayer game w...,"[['Multi-player', 'PvP', 'Online PvP', 'Co-op'...",0,0,0 - 0,[],[],1782210
169,MultiVersus – Technical Test,2022-05-16,0.00,,[[]],0,0,0 - 0,[],[],1829770
...,...,...,...,...,...,...,...,...,...,...,...
83568,Neon Fantasy: Predators,2023-12-14,1.99,Neon Fantasy: Predators is the perfect puzzle ...,"[['Single-player', 'Steam Achievements', 'Stea...",0,0,0 - 0,[],[],2717240
83589,MetaStrike,2024-04-22,0.00,"MetaStrike is a first-person game, multiplayer...","[['Single-player', 'Multi-player', 'PvP', 'Onl...",0,0,0 - 0,[],[],2476720
83594,Klaystar,2022-09-16,0.00,"Klaystar is a 2D precision platformer, where t...",[['Single-player']],0,0,0 - 0,[],[],2091880
83598,临 第一部分“假作真”篇,2023-07-02,0.00,'CONFRONTING'is a science fiction micro horror...,[['Single-player']],0,0,0 - 0,[],[],1565110


Some games just seem to be developer tests. Let's remove them. We'll also remove games with no reviews or no categories:

In [52]:
# Filter games without sales, reviews or categories
df2 = df[~((df['estimated_owners'] == "0 - 0") | (df['positive'] + df['negative'] == 0) | (df['categories'].str.len() == 0))]

To keep things simple, I'll also remove games older than 2013, since there are very few games more than 10 year old on Steam:

In [53]:
# Filter games released before 2013
df2 = df2.copy()
df2['release_date'] = pd.to_datetime(df2['release_date'], format='mixed')
df2 = df2[df2['release_date'].dt.year >= 2013]

Let's see the DataFrame again. It seems we got rid of more than 20000 irrelevant games:

In [54]:
df2

,name,release_date,price,short_description,categories,positive,negative,estimated_owners,tags,tag_frequencies,app_id
1,PUBG: BATTLEGROUNDS,2017-12-21,0.00,Play PUBG: BATTLEGROUNDS for free. Land on str...,"[['Multi-player', 'PvP', 'Online PvP', 'Stats'...",1379580,981860,50000000 - 100000000,"[Survival, Shooter, Battle Royale, Multiplayer...","[14611, 12477, 10683, 10648, 8242, 7598, 7202,...",578080
2,Dota 2,2013-07-09,0.00,"Every day, millions of players worldwide enter...","[['Multi-player', 'Co-op', 'Steam Trading Card...",1832477,406030,200000000 - 500000000,"[Free to Play, MOBA, Multiplayer, Strategy, e-...","[59559, 19966, 15153, 14096, 11631, 10822, 816...",570
3,Grand Theft Auto V,2015-04-13,0.00,Grand Theft Auto V for PC offers players the o...,"[['Single-player', 'Multi-player', 'PvP', 'Onl...",1557234,236827,50000000 - 100000000,"[Open World, Action, Multiplayer, Crime, Autom...","[31998, 23177, 21328, 19059, 18579, 18412, 180...",271590
4,Tom Clancy's Rainbow Six® Siege,2015-12-01,19.99,"Tom Clancy's Rainbow Six® Siege is an elite, t...","[['Single-player', 'Multi-player', 'PvP', 'Onl...",1094330,190046,20000000 - 50000000,"[FPS, PvP, e-sports, Multiplayer, Tactical, Sh...","[9689, 9073, 9026, 8948, 8948, 8944, 8901, 888...",359550
8,Rust,2018-02-08,39.99,The only aim in Rust is to survive. Everything...,"[['Multi-player', 'MMO', 'PvP', 'Online PvP', ...",922081,136729,50000000 - 100000000,"[Survival, Crafting, Multiplayer, Open World, ...","[17867, 11333, 11263, 10699, 8363, 7973, 7266,...",252490
...,...,...,...,...,...,...,...,...,...,...,...
83641,Circuit Breaker,2021-10-21,4.99,"In the cyber-future of 2009, the most popular ...","[['Single-player', 'Family Sharing']]",1,0,0 - 20000,"[Strategy, Casual, PvE, Singleplayer, Tactical...","[72, 48, 47, 44, 41, 37, 32, 29, 27, 25, 22, 2...",1745960
83642,Chess vs Chat,2022-08-09,3.99,Allow your chatters to actively participate an...,"[['Single-player', 'Multi-player', 'PvP', 'Onl...",2,1,0 - 20000,"[Indie, Strategy, Puzzle, Multiplayer, 2D, Che...","[31, 21, 11, 11, 11, 11, 11, 11]",1888920
83643,Orbtangle,2022-10-28,1.99,Orbtangle is a simplistic puzzle game which re...,"[['Single-player', 'Steam Achievements', 'Fami...",5,0,0 - 20000,"[Puzzle, Colorful, Relaxing, Logic, Minimalist...","[54, 46, 44, 41, 38, 33, 30, 27, 25, 24, 22, 2...",2166420
83644,Flashlight,2021-10-25,2.99,"Salvage scrap, build defenses, and rank-up to ...","[['Single-player', 'Multi-player', 'PvP', 'Onl...",3,0,0 - 20000,"[Top-Down Shooter, Rogue-lite, Action Roguelik...","[138, 132, 128, 125, 121, 116, 108, 107, 104, ...",1456260


I'll also split the 'estimated_owners' column into two different variables, this way we'll be able to use it for aggregation in Tableau:

In [55]:
# Split estimated_owners into two: min_owners and max_owners
df2[['min_owners', 'max_owners']] = df2['estimated_owners'].str.split(' - ', expand=True)

# Remove the original field
df2 = df2.drop('estimated_owners', axis=1)

In [56]:
df2[['min_owners', 'max_owners']]

,min_owners,max_owners
1,50000000,100000000
2,200000000,500000000
3,50000000,100000000
4,20000000,50000000
8,50000000,100000000
...,...,...
83641,0,20000
83642,0,20000
83643,0,20000
83644,0,20000


Let's have a look at the distribution of prices:

In [57]:
# Box plot of price
df2.boxplot(column=['price'])

<Axes: >

In [58]:
# Games priced above 200$
df2[df2['price'] > 200]

,name,release_date,price,short_description,categories,positive,negative,tags,tag_frequencies,app_id,min_owners,max_owners
51271,Aartform Curvy 3D 3.0,2013-11-12,299.90,Curvy 3D 3.0 is an innovative sculpting progra...,[['Single-player']],32,13,[Animation & Modeling],[24],253670,0,20000
58416,Ascent Free-Roaming VR Experience,2019-12-27,999.00,A premium highly-immersive VR experience devel...,"[['Single-player', 'Multi-player', 'Co-op', 'L...",6,1,"[Action, Hentai, Nudity, VR, Sexual Content, N...","[88, 33, 26, 26, 24, 23, 21, 17, 17, 16, 12, 1...",1200520,0,20000
64554,Houdini Indie,2018-10-10,269.99,Houdini Indie's procedural node-based workflow...,"[['Partial Controller Support', 'Steam Cloud']]",200,16,"[Animation & Modeling, Game Development, 3D, D...","[64, 60, 42, 39, 27, 22, 16, 7]",502570,0,20000
73341,VEGAS Edit 20 Steam Edition,2022-11-01,249.00,"VEGAS Edit 20 Steam Edition. Fast, easy editin...",[[]],28,11,"[Video Production, Software]","[34, 14]",2070990,0,20000
78274,3DF Zephyr Lite Steam Edition,2016-02-02,249.00,3DF Zephyr allows you to automatically and eas...,[['Steam Trading Cards']],69,2,"[Video Production, Animation & Modeling, Desig...","[33, 33, 32, 22]",438450,0,20000
78722,Aartform Curvy 3D 4.0,2020-01-20,299.99,Curvy 3D 4.0 is an innovative sculpting progra...,[[]],10,2,"[Design & Illustration, Animation & Modeling, 3D]","[22, 21, 10]",1105450,0,20000


We can see that the game priced at $999 is basically a cash-grab without any actual sales, and being an extreme outlier it can distort our analysis. Let's remove it:

In [59]:
# Delete game with id 26936
df2 = df2[df2['app_id'] != 26936]

In [60]:
df2

,name,release_date,price,short_description,categories,positive,negative,tags,tag_frequencies,app_id,min_owners,max_owners
1,PUBG: BATTLEGROUNDS,2017-12-21,0.00,Play PUBG: BATTLEGROUNDS for free. Land on str...,"[['Multi-player', 'PvP', 'Online PvP', 'Stats'...",1379580,981860,"[Survival, Shooter, Battle Royale, Multiplayer...","[14611, 12477, 10683, 10648, 8242, 7598, 7202,...",578080,50000000,100000000
2,Dota 2,2013-07-09,0.00,"Every day, millions of players worldwide enter...","[['Multi-player', 'Co-op', 'Steam Trading Card...",1832477,406030,"[Free to Play, MOBA, Multiplayer, Strategy, e-...","[59559, 19966, 15153, 14096, 11631, 10822, 816...",570,200000000,500000000
3,Grand Theft Auto V,2015-04-13,0.00,Grand Theft Auto V for PC offers players the o...,"[['Single-player', 'Multi-player', 'PvP', 'Onl...",1557234,236827,"[Open World, Action, Multiplayer, Crime, Autom...","[31998, 23177, 21328, 19059, 18579, 18412, 180...",271590,50000000,100000000
4,Tom Clancy's Rainbow Six® Siege,2015-12-01,19.99,"Tom Clancy's Rainbow Six® Siege is an elite, t...","[['Single-player', 'Multi-player', 'PvP', 'Onl...",1094330,190046,"[FPS, PvP, e-sports, Multiplayer, Tactical, Sh...","[9689, 9073, 9026, 8948, 8948, 8944, 8901, 888...",359550,20000000,50000000
8,Rust,2018-02-08,39.99,The only aim in Rust is to survive. Everything...,"[['Multi-player', 'MMO', 'PvP', 'Online PvP', ...",922081,136729,"[Survival, Crafting, Multiplayer, Open World, ...","[17867, 11333, 11263, 10699, 8363, 7973, 7266,...",252490,50000000,100000000
...,...,...,...,...,...,...,...,...,...,...,...,...
83641,Circuit Breaker,2021-10-21,4.99,"In the cyber-future of 2009, the most popular ...","[['Single-player', 'Family Sharing']]",1,0,"[Strategy, Casual, PvE, Singleplayer, Tactical...","[72, 48, 47, 44, 41, 37, 32, 29, 27, 25, 22, 2...",1745960,0,20000
83642,Chess vs Chat,2022-08-09,3.99,Allow your chatters to actively participate an...,"[['Single-player', 'Multi-player', 'PvP', 'Onl...",2,1,"[Indie, Strategy, Puzzle, Multiplayer, 2D, Che...","[31, 21, 11, 11, 11, 11, 11, 11]",1888920,0,20000
83643,Orbtangle,2022-10-28,1.99,Orbtangle is a simplistic puzzle game which re...,"[['Single-player', 'Steam Achievements', 'Fami...",5,0,"[Puzzle, Colorful, Relaxing, Logic, Minimalist...","[54, 46, 44, 41, 38, 33, 30, 27, 25, 24, 22, 2...",2166420,0,20000
83644,Flashlight,2021-10-25,2.99,"Salvage scrap, build defenses, and rank-up to ...","[['Single-player', 'Multi-player', 'PvP', 'Onl...",3,0,"[Top-Down Shooter, Rogue-lite, Action Roguelik...","[138, 132, 128, 125, 121, 116, 108, 107, 104, ...",1456260,0,20000


## Adding game length

The second dataset we'll use contains information about game duration, submitted by users on the website [HowLongToBeat](https://howlongtobeat.com/). Let's load it and have a look:

In [61]:
second_file = 'steamdb.json'
df_extra = pd.read_json(second_file)

In [62]:
df_extra

,sid,store_url,store_promo_url,store_uscore,published_store,published_meta,published_stsp,published_hltb,published_igdb,image,...,meta_url,meta_score,meta_uscore,grnk_score,igdb_url,igdb_single,igdb_complete,igdb_score,igdb_uscore,igdb_popularity
0,10,https://store.steampowered.com/app/10,https://www.youtube.com/watch?v=oKC9SAF4JAc,97.0,2000-11-01,2000-11-08,2000-11-01,1999-06-12,1999-06-12,https://steamcdn-a.akamaihd.net/steam/apps/10/...,...,https://www.metacritic.com/game/pc/counter-str...,88.0,92.0,NaN,https://www.igdb.com/games/counter-strike,NaN,NaN,70.0,83.0,25.74
1,20,https://store.steampowered.com/app/20,None,84.0,1999-04-01,1999-04-07,1999-04-01,1999-04-07,1999-04-07,https://steamcdn-a.akamaihd.net/steam/apps/20/...,...,https://www.metacritic.com/game/pc/team-fortre...,NaN,71.0,NaN,https://www.igdb.com/games/team-fortress-classic,NaN,NaN,NaN,70.0,1.67
2,30,https://store.steampowered.com/app/30,https://www.youtube.com/watch?v=j4MCo89bTWE,90.0,2003-05-01,2003-05-06,2003-05-01,2000-08-04,2003-05-01,https://steamcdn-a.akamaihd.net/steam/apps/30/...,...,https://www.metacritic.com/game/pc/day-of-defe...,79.0,91.0,NaN,https://www.igdb.com/games/day-of-defeat,NaN,NaN,71.0,76.0,1.45
3,40,https://store.steampowered.com/app/40,https://www.youtube.com/watch?v=jN_18uhiMS8,82.0,2001-06-01,2001-06-07,2001-06-01,2001-06-01,2001-07-01,https://steamcdn-a.akamaihd.net/steam/apps/40/...,...,https://www.metacritic.com/game/pc/deathmatch-...,NaN,68.0,NaN,https://www.igdb.com/games/deathmatch-classic,NaN,NaN,NaN,75.0,1.00
4,50,https://store.steampowered.com/app/50,https://www.youtube.com/watch?v=ZNZsWm-Ulk4,95.0,1999-11-01,1999-10-31,1999-11-01,1999-11-01,1999-11-10,https://steamcdn-a.akamaihd.net/steam/apps/50/...,...,https://www.metacritic.com/game/pc/half-life-o...,NaN,86.0,NaN,https://www.igdb.com/games/half-life-opposing-...,6.0,4.0,70.0,82.0,3.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53976,1966310,https://store.steampowered.com/app/1966310,None,NaN,2022-04-22,None,None,None,None,https://cdn.akamai.steamstatic.com/steam/apps/...,...,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
53977,1971880,https://store.steampowered.com/app/1971880,None,NaN,2022-04-22,None,None,2016-12-09,None,https://cdn.akamai.steamstatic.com/steam/apps/...,...,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
53978,1971980,https://store.steampowered.com/app/1971980,None,NaN,2022-04-21,None,None,None,None,https://cdn.akamai.steamstatic.com/steam/apps/...,...,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
53979,1976030,https://store.steampowered.com/app/1976030,None,NaN,2022-04-24,None,None,None,None,https://cdn.akamai.steamstatic.com/steam/apps/...,...,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN


We're only interested in the column 'hltb_single', that contains the information on game length we need. Each Steam game has a unique identifier we can use to join the data from both datasets. This unique identifier is found in the column 'app_id' of the first dataset, and in the column 'sid' of the second dataset. First we'll have to convert 'app_id' to integer since it is currently an object. Let's join the data and see the result:

In [63]:
# Convert 'app_id' integer
df2['app_id'] = pd.to_numeric(df2['app_id'], errors='coerce').astype('Int64')

# Perform a left join for 'hltb_single'
df_merged = pd.merge(df2, df_extra[['sid', 'hltb_single']], left_on='app_id', right_on='sid', how='left')

# Drop the redundant 'sid' column
df_merged.drop('sid', axis=1, inplace=True)

In [64]:
df_merged

,name,release_date,price,short_description,categories,positive,negative,tags,tag_frequencies,app_id,min_owners,max_owners,hltb_single
0,PUBG: BATTLEGROUNDS,2017-12-21,0.00,Play PUBG: BATTLEGROUNDS for free. Land on str...,"[['Multi-player', 'PvP', 'Online PvP', 'Stats'...",1379580,981860,"[Survival, Shooter, Battle Royale, Multiplayer...","[14611, 12477, 10683, 10648, 8242, 7598, 7202,...",578080,50000000,100000000,NaN
1,Dota 2,2013-07-09,0.00,"Every day, millions of players worldwide enter...","[['Multi-player', 'Co-op', 'Steam Trading Card...",1832477,406030,"[Free to Play, MOBA, Multiplayer, Strategy, e-...","[59559, 19966, 15153, 14096, 11631, 10822, 816...",570,200000000,500000000,NaN
2,Grand Theft Auto V,2015-04-13,0.00,Grand Theft Auto V for PC offers players the o...,"[['Single-player', 'Multi-player', 'PvP', 'Onl...",1557234,236827,"[Open World, Action, Multiplayer, Crime, Autom...","[31998, 23177, 21328, 19059, 18579, 18412, 180...",271590,50000000,100000000,31.0
3,Tom Clancy's Rainbow Six® Siege,2015-12-01,19.99,"Tom Clancy's Rainbow Six® Siege is an elite, t...","[['Single-player', 'Multi-player', 'PvP', 'Onl...",1094330,190046,"[FPS, PvP, e-sports, Multiplayer, Tactical, Sh...","[9689, 9073, 9026, 8948, 8948, 8944, 8901, 888...",359550,20000000,50000000,NaN
4,Rust,2018-02-08,39.99,The only aim in Rust is to survive. Everything...,"[['Multi-player', 'MMO', 'PvP', 'Online PvP', ...",922081,136729,"[Survival, Crafting, Multiplayer, Open World, ...","[17867, 11333, 11263, 10699, 8363, 7973, 7266,...",252490,50000000,100000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,Circuit Breaker,2021-10-21,4.99,"In the cyber-future of 2009, the most popular ...","[['Single-player', 'Family Sharing']]",1,0,"[Strategy, Casual, PvE, Singleplayer, Tactical...","[72, 48, 47, 44, 41, 37, 32, 29, 27, 25, 22, 2...",1745960,0,20000,NaN
68202,Chess vs Chat,2022-08-09,3.99,Allow your chatters to actively participate an...,"[['Single-player', 'Multi-player', 'PvP', 'Onl...",2,1,"[Indie, Strategy, Puzzle, Multiplayer, 2D, Che...","[31, 21, 11, 11, 11, 11, 11, 11]",1888920,0,20000,NaN
68203,Orbtangle,2022-10-28,1.99,Orbtangle is a simplistic puzzle game which re...,"[['Single-player', 'Steam Achievements', 'Fami...",5,0,"[Puzzle, Colorful, Relaxing, Logic, Minimalist...","[54, 46, 44, 41, 38, 33, 30, 27, 25, 24, 22, 2...",2166420,0,20000,NaN
68204,Flashlight,2021-10-25,2.99,"Salvage scrap, build defenses, and rank-up to ...","[['Single-player', 'Multi-player', 'PvP', 'Onl...",3,0,"[Top-Down Shooter, Rogue-lite, Action Roguelik...","[138, 132, 128, 125, 121, 116, 108, 107, 104, ...",1456260,0,20000,NaN


In [65]:
# Box plot of hltb_single
df_merged.boxplot(column=['hltb_single'])

<Axes: >

In [66]:
# Games with more than 500 hours of duration
df_merged[df_merged['hltb_single'] > 500]

,name,release_date,price,short_description,categories,positive,negative,tags,tag_frequencies,app_id,min_owners,max_owners,hltb_single
1047,Melvor Idle,2021-11-18,9.99,"Inspired by RuneScape, Melvor Idle takes the c...","[['Single-player', 'Steam Achievements', 'Stea...",10852,1151,"[Idler, RPG, Casual, Indie, Singleplayer, Adve...","[187, 138, 133, 117, 105, 104, 96, 95, 87, 86,...",1267910,500000,1000000,4056.0
1065,NGU IDLE,2019-10-01,0.00,NGU Idle is a Free To Play Idle game that's fu...,"[['Single-player', 'Steam Achievements', 'In-A...",10623,429,"[Free to Play, Idler, Indie, Clicker, Adventur...","[155, 133, 122, 90, 89, 77, 77, 51, 50, 49, 25...",1147690,500000,1000000,5149.0
3773,Zombidle : REMONSTERED,2017-05-30,0.00,"Like the song says, good guys finish last, and...","[['Single-player', 'Cross-Platform Multiplayer...",1441,244,"[Free to Play, Clicker, Casual, Indie, Zombies...","[118, 88, 52, 49, 49, 42, 25, 25, 18, 10, 10]",612020,200000,500000,774.0


Some games are extreme outliers in terms of duration. This is not caused by these games being extremely long but by the fact that some games can be played indefinitely and very few users have reported game length for these types of games. This might distort our analysis, so we'll limit the maximum duration of games at 100 hours, which is a reasonable upper limit for most games:

In [67]:
# Limit game duration to 100 hours
df_merged['hltb_single'] = df_merged['hltb_single'].apply(lambda x: 100 if x > 100 else x)

## Normalizing data

The DataFrame contains fields such as 'categories' or 'tags' that consist on lists of values. To improve the performance of the visualization we'll build in Tableau we must convert this fields into separate tables, that will be linked with the main table using the 'app_id' column.

In [68]:
# Create a separate DataFrame for each list-type column
df_categories = df_merged.explode('categories')[['app_id', 'categories']]
df_tags = df_merged.explode('tags')[['app_id', 'tags']]
df_frequencies = df_merged.explode('tag_frequencies')['tag_frequencies']
df_tags['tag_frequencies'] = df_frequencies.values

# Remove the list columns from the main DataFrame
columns_to_remove = ['categories', 'tags', 'tag_frequencies']
df_imploded = df_merged.drop(columns=columns_to_remove)

In [69]:
df_imploded

,name,release_date,price,short_description,positive,negative,app_id,min_owners,max_owners,hltb_single
0,PUBG: BATTLEGROUNDS,2017-12-21,0.00,Play PUBG: BATTLEGROUNDS for free. Land on str...,1379580,981860,578080,50000000,100000000,NaN
1,Dota 2,2013-07-09,0.00,"Every day, millions of players worldwide enter...",1832477,406030,570,200000000,500000000,NaN
2,Grand Theft Auto V,2015-04-13,0.00,Grand Theft Auto V for PC offers players the o...,1557234,236827,271590,50000000,100000000,31.0
3,Tom Clancy's Rainbow Six® Siege,2015-12-01,19.99,"Tom Clancy's Rainbow Six® Siege is an elite, t...",1094330,190046,359550,20000000,50000000,NaN
4,Rust,2018-02-08,39.99,The only aim in Rust is to survive. Everything...,922081,136729,252490,50000000,100000000,NaN
...,...,...,...,...,...,...,...,...,...,...
68201,Circuit Breaker,2021-10-21,4.99,"In the cyber-future of 2009, the most popular ...",1,0,1745960,0,20000,NaN
68202,Chess vs Chat,2022-08-09,3.99,Allow your chatters to actively participate an...,2,1,1888920,0,20000,NaN
68203,Orbtangle,2022-10-28,1.99,Orbtangle is a simplistic puzzle game which re...,5,0,2166420,0,20000,NaN
68204,Flashlight,2021-10-25,2.99,"Salvage scrap, build defenses, and rank-up to ...",3,0,1456260,0,20000,NaN


The main DataFrame is ready. Since the goal of the dashboard I want to build is to filter data by tags or categories and view their most relevant metrics, I'll remove any categories and tags with less than 50 games, since they are not relevant enough:

In [70]:
# Filter out categories with less than 50 games
categories_counts = df_categories['categories'].value_counts()
categories_to_keep = categories_counts[categories_counts >= 50].index.tolist()
df_categories = df_categories[df_categories['categories'].isin(categories_to_keep)]

In [71]:
df_categories

,app_id,categories
0,578080,['Multi-player'
0,578080,'PvP'
0,578080,'Online PvP'
0,578080,'Stats'
0,578080,'Remote Play on Phone'
...,...,...
68204,1456260,'Family Sharing']
68205,655980,['Single-player'
68205,655980,'Tracked Controller Support'
68205,655980,'VR Only'


In [72]:
# Filter out tags with less than 50 games
tags_counts = df_tags['tags'].value_counts()
tags_to_keep = tags_counts[tags_counts >= 50].index.tolist()
df_tags = df_tags[df_tags['tags'].isin(tags_to_keep)]

In [73]:
df_tags

,app_id,tags,tag_frequencies
0,578080,Survival,14611
0,578080,Shooter,12477
0,578080,Battle Royale,10683
0,578080,Multiplayer,10648
0,578080,FPS,8242
...,...,...,...
68205,655980,Indie,20
68205,655980,Violent,20
68205,655980,Strategy,19
68205,655980,Adventure,19


## Save results as CSV

Finally, we'll save the results as CSV files that we'll import into Tableau:

In [74]:
df_imploded.to_csv('tableau_games.csv', index=False)
df_categories.to_csv('tableau_categories.csv', index=False)
df_tags.to_csv('tableau_tags.csv', index=False)

## View the Dashboard

Head out to Tableau public to see the dashboard I created with this data:

[Steam Tag Explorer](https://public.tableau.com/app/profile/terencicp/viz/steam-games/Dash)

In [75]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('tableau_games.csv', encoding='utf-8')

In [3]:
df.isnull().sum()

App_ID                 0
Game_Name            773
Release_Date           0
Price                  0
Short_Description    337
Positive               0
Negative               0
Min_Owners             0
Max_Owners             0
HLTB_Single            0
dtype: int64

In [4]:
# Replace missing 'short_description' with "No Description Available"
df['Short_Description'] = df['Short_Description'].fillna("No Description Available")

In [5]:
df.isnull().sum()

App_ID                 0
Game_Name            773
Release_Date           0
Price                  0
Short_Description      0
Positive               0
Negative               0
Min_Owners             0
Max_Owners             0
HLTB_Single            0
dtype: int64

In [9]:
df = df.dropna(subset=['Game_Name'])

In [10]:
df.isnull().sum()

App_ID               0
Game_Name            0
Release_Date         0
Price                0
Short_Description    0
Positive             0
Negative             0
Min_Owners           0
Max_Owners           0
HLTB_Single          0
dtype: int64

In [11]:
df.to_csv('tableau_games1.csv', index=False)

In [13]:
# Load the main CSV file (after dropping null values)
df_main = pd.read_csv('tableau_games.csv')

# Load the two other CSV files that also contain 'App_ID'
df_other1 = pd.read_csv('tableau_tags.csv')
df_other2 = pd.read_csv('tableau_categories.csv')

# Identify app_ids present in the updated main file
existing_app_ids = df_main['App_ID'].tolist()

# Filter the other files based on those app_ids
df_other1_filtered = df_other1[df_other1['App_ID'].isin(existing_app_ids)]
df_other2_filtered = df_other2[df_other2['App_ID'].isin(existing_app_ids)]

# Save the filtered CSVs
df_other1_filtered.to_csv('tableau_tags_filtered.csv', index=False)
df_other2_filtered.to_csv('tableau_categories_filtered.csv', index=False)

# Verify the changes
print("Remaining rows in tableau_tags.csv:", len(df_other1_filtered))
print("Remaining rows in tableau_categories.csv:", len(df_other2_filtered))

Remaining rows in tableau_tags.csv: 910623
Remaining rows in tableau_categories.csv: 299597


In [6]:
df = pd.read_csv('csv_categories.csv', encoding='utf-8')

In [9]:
df.isnull().sum()

App_ID        0
Categories    0
dtype: int64

In [8]:
df = df.dropna(subset=['Categories'])

In [10]:
df.to_csv('csv_categories1.csv', index=False)